In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append('/fs/cbsuvlaminck2/workdir/bmg224/hiprfish/image_analysis_code')
import image_functions as imfn

In [21]:
import pandas as pd
import os
import numpy as np

In [ ]:
downsample=2
search_radius=20
image_processing_dir='../image_processing'
# for each sample name
sample_names = imfn.get_sample_names(data_dir=image_processing_dir, sample_glob='*Stitch', ext='.npy')
for sn in sample_names:
    spot_props = pd.DataFrame()
    cell_props = pd.DataFrame()
    spot_split_folder = image_processing_dir + '/' + sn + '_spot_seg_split'
    cell_split_folder = image_processing_dir + '/' + sn + '_cell_seg_split'
    print(spot_split_folder, os.path.exists(spot_split_folder))
    spot_split_fns = imfn.get_sample_names(data_dir=spot_split_folder, sample_glob='*col_[012]', ext='.npy')
    cell_split_fns = imfn.get_sample_names(data_dir=cell_split_folder, sample_glob='*col_[012]', ext='.npy')
    spot_split_fns = [spot_split_folder + '/' + i for i in spot_split_fns]
    cell_split_fns = [cell_split_folder + '/' + i for i in cell_split_fns]
    # for each split file
    for sfn, cfn in zip(spot_split_fns, cell_split_fns):
        ss_fn = sfn + '_seg.npy'
        sr_fn = sfn + '.npy'
        cs_fn = cfn + '_seg.npy'
        cr_fn = cfn + '.npy'
        # Load cell and spot numpy segs and raw
        ss = np.load(ss_fn)
        cs = np.load(cs_fn)
        sr = np.load(sr_fn)
        cr = np.load(cr_fn)
        # Get spot and cell object props
        sp = imfn.measure_regionprops(ss, sr)
        cp = imfn.measure_regionprops(cs, cr)
        # Get spot cell ids
        sp = imfn.spots_to_cell_id(cs, 
                                   sp,
                                   search_radius, 
                                   downsample)
#         print(sp.shape, sp.columns)
        # add unique identifier to object id
        print(spot_props.shape[0])
        sp_max_label = spot_props.ID.max() if spot_props.shape[0] > 0 else 0
        cp_max_label = cell_props.ID.max() if cell_props.shape[0] > 0 else 0
        sp['id_unique'] = sp.ID + sp_max_label
        cp['id_unique'] = cp.ID + cp_max_label
        # merge props with sample name csv    
        spot_props = spot_props.append(sp, ignore_index=True)        
        cell_props = cell_props.append(cp, ignore_index=True)
    output_sp_fn = image_processing_dir + '/' + sn + '_spot_seg_cell_id.csv'
    output_cp_fn = image_processing_dir + '/' + sn + '_cell_seg_props.csv'
    spot_props.to_csv(output_sp_fn)
    cell_props.to_csv(output_cp_fn)
    spot_props = pd.read_csv(output_sp_fn)
    print(spot_props.shape, spot_props.columns)

../image_processing/2021_06_02_plasmidairyscan_exp_080220_method_b_plasmid_pos_fov_tile3_Airyscan_Processing_Stitch_spot_seg_split True
0
9648
15407
19217
24547
30443
39302
45430
54177
(62786, 12) Index(['Unnamed: 0', 'ID', 'X', 'Y', 'Area', 'Intensity', 'ParentCellID',
       'major_axis_length', 'minor_axis_length', 'cell_id', 'dist',
       'id_unique'],
      dtype='object')
../image_processing/2021_06_02_plasmidairyscan_exp_040421_method_f_plasmid_pos_fov_tile2_Airyscan_Processing_Stitch_spot_seg_split True
0
8956
14908
20653
29364
37965
44280
49913
53503
(62178, 12) Index(['Unnamed: 0', 'ID', 'X', 'Y', 'Area', 'Intensity', 'ParentCellID',
       'major_axis_length', 'minor_axis_length', 'cell_id', 'dist',
       'id_unique'],
      dtype='object')
../image_processing/2021_06_02_plasmidairyscan_exp_080220_method_b_plasmid_pos_fov_tile1_Airyscan_Processing_Stitch_spot_seg_split True
0
8326
14042
17563
26596
35518
40841
47002
55255
(61519, 12) Index(['Unnamed: 0', 'ID', 'X', 'Y', 'A

In [10]:
cell_props.shape

(0, 0)